In [ ]:
corpus = [
    "SELECT * FROM customers WHERE active = 1;",
    "INSERT INTO products (name, price) VALUES ('Laptop', 1200);",
    "UPDATE employees SET salary = 5000 WHERE department = 'HR';",
    "DELETE FROM orders WHERE order_date < '2023-01-01';",
    "CREATE TABLE users (id INT PRIMARY KEY, username VARCHAR(50));",
    "ALTER TABLE invoices ADD COLUMN total DECIMAL(10, 2);",
    "DROP TABLE IF EXISTS temp_data;",
    "SELECT name, COUNT(*) AS total FROM sales GROUP BY name;",
    "JOIN addresses ON customers.id = addresses.customer_id;",
    "ORDER BY created_at DESC LIMIT 10;",
    "WHERE status = 'pending' AND priority > 5;",
    "HAVING SUM(amount) > 1000;",
    "GRANT SELECT, INSERT ON database.* TO 'user'@'localhost';",
    "REVOKE ALL PRIVILEGES ON employees FROM 'intern'@'%';",
    "BEGIN TRANSACTION; COMMIT;",
    "ROLLBACK TO SAVEPOINT before_update;",
    "EXPLAIN SELECT * FROM logs WHERE type = 'error';",
    "INDEX idx_name ON employees (last_name);",
    "PRIMARY KEY (order_id, product_id);",
    "FOREIGN KEY (customer_id) REFERENCES customers(id);"
]

# Pré-processamento

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter

# Tokenização e vocabulário
tokens = [word.lower() for sentence in corpus for word in sentence.split()]
vocab = Counter(tokens)
vocab = sorted(vocab, key=vocab.get, reverse=True)
vocab_size = len(vocab)

# Mapeamento palavra para índice
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}

# Tarefa 1: Modelo de linguagem

Nesta etapa, vamos criar o modelo de linguagem que será treinado para prever a próxima palavra com base no contexto das palavras anteriores. O modelo será uma rede neural baseada em embeddings e LSTMs, que são adequados para lidar com sequências de texto.

---

### **Estrutura do Modelo**

O modelo será composto por:

1. **Camada de Embedding**  
   - Converte os índices das palavras em vetores densos de tamanho fixo (representação vetorial das palavras).  
   - Tamanho do embedding (`embedding_dim`) é um hiperparâmetro.

2. **Camada LSTM**  
   - Processa as sequências de embeddings e mantém informações sobre o contexto anterior.  
   - Tamanho do estado oculto (`hidden_dim`) é outro hiperparâmetro.

3. **Camada Linear (Fully Connected)**  
   - Projeta a saída da LSTM para o espaço do vocabulário, gerando scores para cada palavra no vocabulário.

4. **Softmax (aplicado durante inferência)**  
   - Converte os scores para probabilidades de cada palavra no vocabulário.

# Tarefa 2: Dataset e DataLoader

Nesta tarefa, vamos criar um **Dataset** e um **DataLoader** para alimentar o modelo com dados durante o treinamento. O Dataset será responsável por preparar as sequências de entrada e saída a partir do corpus, enquanto o DataLoader organizará os dados em lotes (`batches`) para treinamento eficiente.

---

### **Estrutura do Dataset**

O Dataset utilizará o corpus pré-processado e fará o seguinte:

1. **Divisão em Sequências**  
   - Cada sentença será dividida em sequências de comprimento fixo (`seq_length`).  
   - Por exemplo, dado o texto: `"SELECT * FROM customers"`, com `seq_length=3`:
     - Entrada (`x`): `"SELECT * FROM"`
     - Saída (`y`): `"* FROM customers"`

2. **Mapeamento para Índices**  
   - Cada palavra será convertida para seu índice correspondente no vocabulário (`word_to_idx`).

3. **Par Entrada-Saída**  
   - Para cada sequência de entrada (`x`), haverá uma sequência de saída (`y`), que corresponde às próximas palavras.

---


# Tarefa 3: Treinamento

Agora que temos o **modelo**, o **Dataset** e o **DataLoader**, o próximo passo é implementar o loop de treinamento. O objetivo do treinamento é ajustar os pesos do modelo para minimizar a perda (loss), que mede a diferença entre as previsões do modelo e as saídas reais.

---

### **Etapas do Treinamento**

1. **Definir Função de Perda e Otimizador**  
   - A função de perda será a `CrossEntropyLoss`, que é adequada para problemas de classificação, como prever a próxima palavra.  
   - O otimizador será o `Adam`, que é eficiente para redes neurais profundas.

2. **Loop de Treinamento**  
   - Para cada época:
     - Iterar sobre os lotes do DataLoader.
     - Passar o lote pelo modelo para obter as previsões.
     - Calcular a perda comparando as previsões com as saídas reais.
     - Atualizar os pesos do modelo usando o otimizador.

3. **Monitorar o Progresso**  
   - Exibir a perda a cada época para acompanhar o aprendizado do modelo.



# Função de completar texto

In [ ]:
def complete_text(seed_text, num_words=5, temperature=0.7):
    model.eval()
    words = seed_text.lower().split()

    with torch.no_grad():
        for _ in range(num_words):
            inputs = torch.tensor(
                [word_to_idx[word] for word in words[-3:]]  # Usa contexto de 3 palavras
            ).unsqueeze(0).to(device)

            output, _ = model(inputs)
            probabilities = torch.softmax(output[0, -1] / temperature, dim=0)
            next_idx = torch.multinomial(probabilities, 1).item()

            words.append(idx_to_word[next_idx])

    return " ".join(words).capitalize()

In [ ]:
# Exemplos de prompts
print(complete_text("SELECT * FROM", num_words=5))
print(complete_text("ADD COLUMN total", num_words=3))
print(complete_text("ROLLBACK TO SAVEPOINT", num_words=4))